In [4]:
from nltk.tokenize.casual import casual_tokenize
import gensim
from gensim import corpora
import re
import pandas as pd
from string import punctuation
from gensim.models import CoherenceModel
import os

input_filepath="../../data/processed/"
filename="tweets_w_lga.csv"

In [5]:


tweetdf=pd.read_csv(os.path.join(input_filepath, filename),header=0)
tweetdf

,Unnamed: 0,Unnamed: 0.1,id,time,created_at,from_user_name,text,filter_level,possibly_sensitive,withheld_copyright,...,from_user_timezone,from_user_lang,from_user_tweetcount,from_user_followercount,from_user_friendcount,from_user_favourites_count,from_user_listed,from_user_withheld_scope,from_user_created_at,lga
0,0,0,951300429454831616,1515642671,2018-01-11 03:51:11,patientsafe3,@kaye_rolls @DrUmeshPrabhu @alisonleary1 @Matt...,low,NaN,NaN,...,NaN,en,27386,3790,4998,12523,117,NaN,2016-01-23 02:54:35,NaN
1,1,1,951300448031473664,1515642675,2018-01-11 03:51:15,Markfchris,Things to do on hot day in Sydney. @ Carriagew...,low,0.0,NaN,...,Sydney,en,14953,695,1725,2615,62,NaN,2010-04-22 02:37:40,Sydney
2,2,2,951300464129093632,1515642679,2018-01-11 03:51:19,JasonStegeman,Harry probably wants his outfit back 😂🤠 https:...,low,0.0,NaN,...,Sydney,en,7431,2263,2880,1029,150,NaN,2014-03-11 06:59:31,NaN
3,3,3,951300475034386432,1515642682,2018-01-11 03:51:22,RyanWilksPtyLtd,@RyanWilksPtyLtd has completed a #HVAC replace...,low,0.0,NaN,...,Sydney,en,48,30,71,1,0,NaN,2012-06-26 23:52:20,NaN
4,4,4,951300560812113920,1515642702,2018-01-11 03:51:42,ZKII_,BB And The Blips x The Red Rattler Theater @ T...,low,0.0,NaN,...,NaN,en,604,86,275,134,3,NaN,2012-10-02 09:55:23,Inner West
5,5,5,951300564435922944,1515642703,2018-01-11 03:51:43,ldeibe,@matonfender https://t.co/BF1Bp8z2b9,low,0.0,NaN,...,Hawaii,en,3769,1076,452,522,45,NaN,2009-09-01 05:32:16,NaN
6,6,6,951300578633687040,1515642706,2018-01-11 03:51:46,ladonnarama,@AaronManhattan https://t.co/kz4bzfJ7mu,low,0.0,NaN,...,Sydney,en,6365,534,254,3690,33,NaN,2009-02-08 01:20:25,NaN
7,7,7,951300629212925953,1515642718,2018-01-11 03:51:58,MaskedNozza,I finished 2 days of of my workout regime in a...,low,NaN,NaN,...,Sydney,en,1380,92,210,537,7,NaN,2014-01-30 01:47:42,NaN
8,8,8,951300630231969792,1515642719,2018-01-11 03:51:59,lmeyrick,@LagermenCC No one is... it would be unfair to...,low,NaN,NaN,...,Sydney,en,12160,1821,983,4012,37,NaN,2009-08-13 23:25:51,NaN
9,9,9,951300666617573376,1515642727,2018-01-11 03:52:07,jessethanley,@moeamaya @AustinGrandt The exposure and exper...,low,NaN,NaN,...,Sydney,en,9278,2486,139,26535,228,NaN,2009-10-19 07:37:16,NaN


In [6]:
#now split by various doc types and run LGA. DO i need to write out interim files? Not really, not yet

by_lga = tweetdf.groupby('lga', as_index=False).agg({'text':lambda x:' '.join(x)})
by_lga['text'] = by_lga.apply(lambda x: casual_tokenize(x['text'], strip_handles=True, reduce_len=True, preserve_case=False), axis=1)
by_lga

,lga,text
0,Bayside,"[i'm, at, westfield, eastgardens, -, in, eastg..."
1,Blacktown,"[awesome, day, at, the, sydney, international,..."
2,Blue Mountains,"[i'm, at, blaxland, in, blaxland, ,, nsw, http..."
3,Burwood,"[green, mango, juice, :, 1, small, green, mang..."
4,Camden,"[now, that, ’, s, some, impressive, yet, simpl..."
5,Campbelltown,"[summer, somedays, ,, jan, 12, @, campbelltown..."
6,Canada Bay,"[shopping, with, my, bff, makes, me, very, hun..."
7,Canterbury-Bankstown,"[latte, . ., #latte, #coffee, #roamingdodos, @..."
8,Central Coast,"[testing, out, his, new, camera, he, got, from..."
9,Cumberland,"[don, ’, t, worry, ,, i, actually, went, to, t..."


In [7]:
by_lga.apply(lambda x: len(x['text']), axis=1)

0      26328
1      14316
2       2332
3       2370
4       4264
5       4460
6       6842
7       7777
8       1123
9      13711
10      4423
11      6614
12      2944
13     10801
14       600
15     49038
16      4344
17      2176
18      8023
19      9310
20     13458
21     55153
22     27746
23     12404
24     46354
25     25575
26      1912
27     23656
28    547823
29      9829
30     50804
31    120661
32       850
33    124512
dtype: int64

In [25]:
punc_regex = re.compile('[%s]' % re.escape(punctuation))
def clean_toks(tokens):
    #regex replace
    tokens = [punc_regex.sub(" ", x.strip("'")) for x in tokens if not re.match(r"^http\S+$", x)]
    tokens = list(filter(len, tokens))
    return tokens

#finds best ntopics
def run_lda(text_series, start, step, stop):
    toks = [x for x in text_series] # join into a 2d list
    toks = list(map(clean_toks,toks)) 

    dictionary = corpora.Dictionary(toks)

    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in toks]

    Lda = gensim.models.ldamodel.LdaModel

    # Running and Trainign LDA model on the document term matrix.
    #ldamodel = Lda(doc_term_matrix, num_topics=num_topics, id2word = dictionary, passes=50)
    #return ldamodel
    model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=toks, start=start, limit=stop, step=step)
    return model_list, coherence_values



NameError: name 'compute_coherence_values' is not defined

In [18]:

toks = [x for x in by_lga['text']] # join into a 2d list
toks = list(map(clean_toks,toks)) 

dictionary = corpora.Dictionary(toks)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in toks]
doc_term_matrix

[[(0, 3810),
  (1, 60),
  (2, 121),
  (3, 1),
  (4, 4),
  (5, 2),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 2),
  (15, 2),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 4),
  (27, 2),
  (28, 1),
  (29, 4),
  (30, 1),
  (31, 5),
  (32, 3),
  (33, 1),
  (34, 3),
  (35, 4),
  (36, 15),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 2),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 7),
  (55, 3),
  (56, 1),
  (57, 1),
  (58, 2),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 2),
  (63, 1),
  (64, 1),
  (65, 4),
  (66, 1),
  (67, 2),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 2),
  (73, 1),
  (74, 1),
  (75, 2),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 1),
  (89, 1),
  (90, 4),
  

NameError: name 'coh' is not defined

In [21]:
## reference: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#17howtofindtheoptimalnumberoftopicsforlda

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=50)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
start, step, stop = (5, 2, 40)
ldams, coh = run_lda(by_lga['text'], start, step, stop)



In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
x = range(start, stop, step)
plt.plot(x, coh)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()